In [1]:
from langchain.llms import Bedrock
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory
import boto3
from IPython.display import display, Markdown


def get_bedrock_client():
    bedrock_client = boto3.client("bedrock-runtime", region_name="us-east-1")
    return bedrock_client

# Instantiate LLM with Cohere Model
llm = Bedrock(
    client=get_bedrock_client(),
    model_id="cohere.command-text-v14",
)

conversation = ConversationChain(
    llm=llm, verbose=True, memory=ConversationBufferMemory() 
)

cohere_output = conversation.predict(input="What can you tell me about the right to asylum in Turkey?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: What can you tell me about the right to asylum in Turkey?
AI:

> Finished chain.


In [2]:
# Instantiate LLM with Claude Model
llm = Bedrock(
    client=get_bedrock_client(),
    model_id="anthropic.claude-v2"
)

conversation = ConversationChain(
    llm=llm, verbose=True, memory=ConversationBufferMemory()
)

claude_output = conversation.predict(input="What can you tell me about the right to asylum in Turkey?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: What can you tell me about the right to asylum in Turkey?
AI:

> Finished chain.


In [3]:
display(Markdown("## Base LLM Output (no Training or RAG): \n **Cohere Command v14**  \n\n" + cohere_output + "\n\n **Claude v2** \n\n" + claude_output))

## Base LLM Output (no Training or RAG): 
 **Cohere Command v14**  

 Turkey is a signatory of the United Nations (UN) Convention relating to the Status of Refugees, its legal framework establishes the rights of refugees and the responsibilities of protecting powers, such as Turkey. 

Turkey offers the right to seek asylum to anyone who can enter its territory, regardless of their nationality or ethnicity. Once in Turkey, the right to asylum grants refugees access to education, legal residence, and access to public services such as healthcare.

In 2015, Turkey adopted a 'geographic limitation' approach to its asylum legislation, stating that international protection can only be granted to those who have fled war zones or neighbouring countries to Turkey. This means that asylum claims from other nationalities are typically not accepted. 

Turkey has been criticized for its treatment of refugees in the past. In particular, there have been reports of poor conditions in refugee camps and detention centers. The Turkish government has also been accused of forcibly returning refugees to war zones and carrying out pushbacks at the border. 

It's important to note that the information available on the topic of asylum rights in Turkey may not be fully up to date, and policies may have changed since the last update. 

Would you like to know more about the asylum process in Turkey? 

 **Claude v2** 

 Here are some key facts about the right to asylum in Turkey:

- Turkey is a party to the 1951 Refugee Convention and its 1967 Protocol. This means Turkey has legal obligations under international law to protect refugees.

- The Turkish constitution also guarantees the right to asylum. Article 90 states that in the case of conflict between international agreements and domestic laws, the international agreements shall prevail.

- The main national legislation dealing with refugees and asylum seekers is the Law on Foreigners and International Protection adopted in 2013. This law lays out provisions for different types of international protection, including refugee status.

- Turkey maintains a geographical limitation to the Refugee Convention - only Europeans can be given refugee status. Non-Europeans are given "conditional refugee" status or subsidiarity protection status.

- In recent years, Turkey has received large numbers of asylum seekers, mainly from Syria, Iraq, Afghanistan, and Iran. As of 2018, Turkey hosted over 3.5 million registered Syrian refugees, the largest refugee population in the world. 

- The United Nations High Commissioner for Refugees (UNHCR) is active in Turkey and works with the Turkish government to process asylum applications and provide services. But UNHCR has faced challenges in